In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

from sklearn.model_selection import train_test_split

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

import pandas as pd

from collections import Counter
from konlpy.tag import Mecab

import time

import numpy as np

In [2]:
import warnings 
warnings.simplefilter('ignore')

In [3]:
df_train = pd.read_csv("./naver_train.csv", encoding="utf-8-sig")
df_test = pd.read_csv("./naver_test.csv", encoding="utf-8-sig")

In [4]:
df_train.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터 보고 초딩 영화 줄오버 연기조차 가볍지 않구나,1
2,10265843,너무 재밓었다 그래서 보는 것을 추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬 페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어 보이기만 했던 커스...,1


In [5]:
def delete_null(data):
    data = data.dropna(axis=0).reset_index(drop=True)
    space_idx = []
    for i in range(len(data)):
        if str.isspace(data.iloc[i, 1]) == True:
            space_idx.append(i)
    data = data.drop(space_idx)
    
    return data

In [6]:
df_train, df_test = map(delete_null, [df_train, df_test])

In [7]:
trainset = np.array(df_train.drop(["id"], axis = 1))
testset = np.array(df_test.drop(["id"], axis = 1))

In [8]:
trainset, valset= train_test_split(trainset, test_size=0.1)

In [9]:
X_train = trainset[:, 0]
y_train = trainset[:, 1]
X_val = valset[:, 0]
y_val = valset[:, 1]
X_test = testset[:, 0]
y_test = testset[:, 1]

In [10]:
y_train = y_train.astype(np.int64)
y_val = y_val.astype(np.int64)
y_test = y_test.astype(np.int64)

In [11]:
class Vocabulary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
        
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

In [12]:
m = Mecab("C:\mecab\mecab-ko-dic")

def tokenizer(text):
    return m.morphs(text)

def build_vocab(data, threshold):
    counter = Counter()

    for i in range(len(data)):
        tokens = tokenizer(data[i])
        counter.update(tokens)

    words = [word for word, cnt in counter.items() if cnt >= threshold]
    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<unk>')
    for w in words:
        vocab.add_word(w)
    return vocab

def tokenizing(data, max_length=256):
    data_size = len(data)
    length = []
    for i in range(data_size):
        data[i] = tokenizer(data[i])
        length.append(len(data[i]))
        if len(data[i]) > max_length:
            data[i] = data[:max_length]
        else:
            for _ in range(max_length-len(data[i])):
                data[i].append("<pad>")
    return data, length

In [13]:
vocab = build_vocab(X_train, 1)

In [14]:
print(vocab.word2idx)

{'<pad>': 0, '<unk>': 1, '재미없': 2, '음': 3, '이게': 4, '재밌': 5, '다는': 6, '분': 7, '들': 8, '은': 9, '스릴러': 10, '다운': 11, '영화': 12, '못': 13, '본': 14, '분임': 15, '내일': 16, '도': 17, '칸타빌레': 18, '를': 19, '보다': 20, '가': 21, '노다메': 22, '와': 23, '치': 24, '아': 25, '키': 26, '의': 27, '케미': 28, '다시': 29, '보': 30, '고': 31, '싶': 32, '어서': 33, '일': 34, '동안': 35, '드라마': 36, '부터': 37, '극장판': 38, '까지': 39, '정': 40, '주행': 41, '햇': 42, '다': 43, '역시': 44, '봐도': 45, '너무': 46, '또': 47, '볼': 48, '수': 49, '있': 50, '는': 51, '치아키': 52, '잘': 53, '어울리': 54, '하나': 55, '어색': 56, '하': 57, '지': 58, '아직': 59, '여운': 60, '이': 61, '남': 62, 'ㅠㅠ': 63, '내용': 64, '괜찮': 65, '이것': 66, '일본': 67, '에서': 68, '실제로': 69, '일어난': 70, '라니': 71, '믿': 72, '으': 73, '없': 74, '진심': 75, '노': 76, '잼': 77, '마': 78, '라': 79, '시간': 80, '아깝': 81, '평점': 82, '호대': 83, '출동': 84, '아니': 85, '되': 86, '오': 87, '점': 88, '으로': 89, '을': 90, '넘': 91, '못하': 92, '이유': 93, '모르': 94, '겠': 95, '시리즈': 96, '에': 97, '비해서': 98, '부족': 99, '한': 100, '많': 101, '지만': 102, '여전히

In [15]:
X_train, X_val, X_test = map(tokenizing, [X_train, X_val, X_test])

In [16]:
print(X_train[0][0])

['재미없', '음', '이게', '재밌', '다는', '분', '들', '은', '스릴러', '다운', '영화', '못', '본', '분임', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '

In [17]:
print(X_train[1][0])

14


In [18]:
def token2idx(data, vocab):
    data_size = len(data[0])
    sentence_length = len(data[0][0])
    for i in range(data_size):
        data[0][i] = [vocab(x) for x in data[0][i]]
    return data

In [19]:
X_train, X_val, X_test = map(lambda data : token2idx(data, vocab), [X_train, X_val, X_test])

In [20]:
print(X_train[0][0])

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [22]:
print(X_train[1][0])

14


In [29]:
class CustomDataset(Dataset):
    def __init__(self, data, y,):
        self.x = data[0]
        self.length = data[1]
        self.y = y
        
    def __getitem__(self, index):
        return (torch.tensor(self.x[index]), self.length[index], self.y[index])
    
    def __len__(self):
        return len(self.x)

In [30]:
trainset = CustomDataset(X_train, y_train)
valset = CustomDataset(X_val, y_val)
testset = CustomDataset(X_test, y_test)

In [31]:
trainset[0]

(tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,

In [32]:
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
valloader = DataLoader(valset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)

In [33]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu 와 cuda 중 다음 기기로 학슴함: ", DEVICE)

cpu 와 cuda 중 다음 기기로 학슴함:  cuda


In [34]:
input_file = "glove.txt"
output_file = "tmp.txt"

glove2word2vec(input_file, output_file)

glove = KeyedVectors.load_word2vec_format(output_file, binary=False)

In [35]:
vocab_size = len(vocab.word2idx.keys())
embedding_size = 100
embedding_weight = np.zeros((vocab_size, embedding_size))
for i in range(2, vocab_size):
    if vocab.idx2word[i] in glove.key_to_index.keys():
        embedding_weight[i] = glove[vocab.idx2word[i]]
embedding_weight = torch.tensor(embedding_weight)

In [36]:
print(embedding_weight)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0018,  0.0581,  0.4003,  ..., -0.2584,  0.7097,  0.1341],
        ...,
        [-0.8084, -1.0427, -0.6031,  ...,  1.0148,  0.3762, -0.0272],
        [-0.9672,  1.0484,  0.1031,  ...,  0.9369, -0.1019,  0.1828],
        [ 0.0614,  0.1266,  0.0806,  ...,  0.4683,  0.3122,  0.3870]],
       dtype=torch.float64)


In [37]:
class LSTM(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
        super(LSTM, self).__init__()
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=n_layers, batch_first= True,  bidirectional=True)
        self.out = nn.Linear(hidden_dim*2, n_classes, bias=True)

    def forward(self, x, length):
        embeded = self.embed(x)
        packed_input = pack_padded_sequence(embeded, length.tolist(), batch_first=True, enforce_sorted=False)
        packed_output,(hidden, cell) = self.lstm(packed_input)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        logit = self.out(hidden)
        return logit

In [38]:
n_classes = 2
model = LSTM(3, 256, vocab_size, 100, n_classes).to(DEVICE)
lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [39]:
def train(model, optimizer, train_iter):
    model.train()
    corrects, total_loss = 0, 0
    size = 0
    for b, batch in enumerate(train_iter):
        x , l, y = batch
        x = x.to(DEVICE)
        y = y.long().to(DEVICE)
        y = y.reshape(-1)
        optimizer.zero_grad()
        logit = model(x, l)
        loss = F.cross_entropy(logit, y, reduction="sum")
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
        size += x.shape[0]
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy 

In [40]:
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    size = 0
    with torch.no_grad():
        for batch in val_iter:
            x , l, y = batch
            x = x.to(DEVICE)
            y = y.long().to(DEVICE)
            y = y.reshape(-1)
            logit = model(x, l)
            loss = F.cross_entropy(logit, y, reduction="sum")
            total_loss += loss.item()
            corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()    
            size += x.shape[0]
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [41]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [42]:
model.embed.weight.data.copy_(embedding_weight)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0018,  0.0581,  0.4003,  ..., -0.2584,  0.7097,  0.1341],
        ...,
        [-0.8084, -1.0427, -0.6031,  ...,  1.0148,  0.3762, -0.0272],
        [-0.9672,  1.0484,  0.1031,  ...,  0.9369, -0.1019,  0.1828],
        [ 0.0614,  0.1266,  0.0806,  ...,  0.4683,  0.3122,  0.3870]],
       device='cuda:0')

In [43]:
best_val_loss = None
n_epochs = 15
for epoch in range(n_epochs):
    
    start_time = time.time()
    
    train_loss, train_accuracy = train(model, optimizer, trainloader)
    val_loss, val_accuracy = evaluate(model, valloader)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_accuracy:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_accuracy:.2f}%')
    
    if not best_val_loss or val_loss < best_val_loss:
        torch.save(model.state_dict(), "./textclassificatior.pt")
        best_val_loss = val_loss

Epoch: 01 | Epoch Time: 0m 47s
	Train Loss: 0.435 | Train Acc: 79.65%
	 Val. Loss: 0.433 |  Val. Acc: 80.03%
Epoch: 02 | Epoch Time: 0m 45s
	Train Loss: 0.364 | Train Acc: 83.70%
	 Val. Loss: 0.356 |  Val. Acc: 83.95%
Epoch: 03 | Epoch Time: 0m 44s
	Train Loss: 0.333 | Train Acc: 85.47%
	 Val. Loss: 0.338 |  Val. Acc: 85.10%
Epoch: 04 | Epoch Time: 0m 45s
	Train Loss: 0.311 | Train Acc: 86.62%
	 Val. Loss: 0.326 |  Val. Acc: 85.75%
Epoch: 05 | Epoch Time: 0m 45s
	Train Loss: 0.291 | Train Acc: 87.58%
	 Val. Loss: 0.346 |  Val. Acc: 84.76%
Epoch: 06 | Epoch Time: 0m 46s
	Train Loss: 0.274 | Train Acc: 88.53%
	 Val. Loss: 0.316 |  Val. Acc: 86.09%
Epoch: 07 | Epoch Time: 0m 46s
	Train Loss: 0.259 | Train Acc: 89.34%
	 Val. Loss: 0.319 |  Val. Acc: 86.08%
Epoch: 08 | Epoch Time: 0m 47s
	Train Loss: 0.244 | Train Acc: 90.11%
	 Val. Loss: 0.324 |  Val. Acc: 86.29%
Epoch: 09 | Epoch Time: 0m 47s
	Train Loss: 0.229 | Train Acc: 90.79%
	 Val. Loss: 0.340 |  Val. Acc: 86.04%
Epoch: 10 | Epoch T

In [44]:
model.load_state_dict(torch.load("./textclassificatior.pt"))

<All keys matched successfully>

In [45]:
test_loss, test_accuracy = evaluate(model, testloader)
print(test_accuracy)

tensor(86.3185, device='cuda:0')
